In [0]:
import tensorflow as tf
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Input,SpatialDropout1D, Embedding, LSTM, Dense, merge, Convolution2D, Lambda, GRU, TimeDistributed, Reshape, Permute, Convolution1D, Masking, Bidirectional
from keras.optimizers import Adam
from keras.layers import concatenate
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras import optimizers, callbacks

Using TensorFlow backend.


In [0]:
from tensorflow.python.client import device_lib
import gzip
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5313538722725207883
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1064455671150370781
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6214015562538571622
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14892338381
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10046645237816242416
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [0]:
!git clone https://github.com/Arpita33/StrongILSData.git
!git clone https://github.com/Arpita33/WeakILSDataset.git

Cloning into 'StrongILSData'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8 (delta 1), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.
Cloning into 'WeakILSDataset'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 1), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (4/4), done.


In [0]:
import numpy as np

In [0]:
gt_strongILS_whole = np.load("StrongILSData/StrongILS_Dataset/valid_tree.npy")
gt_strongILS_taxa_removed = np.load("StrongILSData/StrongILS_Dataset/test_tree.npy")

gt_weakILS_whole = np.load("WeakILSDataset/weak_ILS_train_tree.npy")
gt_weakILS_taxa_removed = np.load("WeakILSDataset/weak_ILS_valid_tree.npy")

gt_total_whole = np.concatenate((gt_strongILS_whole, gt_weakILS_whole), axis=0, out=None)
gt_total_taxa_removed = np.concatenate((gt_strongILS_taxa_removed,gt_weakILS_taxa_removed),axis=0, out=None)

#gt_total_whole.shape

Train_data_X_arr = gt_total_taxa_removed
Train_data_Y_arr = gt_total_whole

print(Train_data_Y_arr.shape)

(7785, 165, 3)


In [0]:
#np.sum(Train_data_Y_arr, axis=-1).all()

In [0]:
x = Train_data_X_arr[2][:40]
#y = Train_data_Y_arr[0][:40]

print(x)

[[0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 0.]]


In [0]:
Train_data_X_reshaped = Train_data_X_arr.reshape(Train_data_X_arr.shape[0],Train_data_X_arr.shape[1]*Train_data_X_arr.shape[2])

Train_data_Y_reshaped = Train_data_Y_arr.reshape(Train_data_Y_arr.shape[0],Train_data_Y_arr.shape[1]*Train_data_Y_arr.shape[2])

In [0]:
print(Train_data_X_reshaped.shape)

#print(Train_data_Y_reshaped.shape)

#print(Train_data_X_reshaped[0][0:20])

#print(Train_data_Y_reshaped[0][0:20])

(7785, 495)


In [0]:
def array_splitter(arr):
  temp = []
  size_of_arr = int(arr.shape[0]//3)
  print(size_of_arr)
  for i in range(1,size_of_arr+1):
    t = arr[3*(i-1) : 3*i]
    #print(t)
    temp.append(t)
    #print(3*(i-1), 3*i)
  return np.asarray(temp)

In [0]:
from keras.layers.core import Reshape

def array_splitter(arr):
  return arr.reshape(int(arr.shape[0]//3),3)

In [0]:
import numpy as np
a = np.zeros([495])
array_splitter(a)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0

In [0]:
from keras.layers import Embedding, Add, concatenate, Input, Softmax
from keras import backend as K
#keras.activations.softmax(x, axis=-1)
from keras.activations import softmax


def get_model(input_size=495):
  
  #https://blog.keras.io/building-autoencoders-in-keras.html
  
  main_input = Input(batch_shape=(None, input_size), name='main_input')
  
  enc1 = Dense(int(input_size//1.5), activation='relu', use_bias=True)(main_input)
  enc2 = Dense(int(input_size//2), activation='relu', use_bias=True)(enc1)
  enc3 = Dense(int(input_size//2.5), activation='relu', use_bias=True)(enc2)
  enc4 = Dense(int(input_size//3),activation='relu', use_bias=True)(enc3)
    
  dec0 = Dense(int(input_size//2.5),activation='relu', use_bias=True)(enc4)
  dec1 = Dense(int(input_size//2), activation='relu', use_bias=True)(dec0)
  dec2 = Dense(int(input_size//1.5), activation='relu', use_bias=True)(dec1)
  dec_output = Dense(input_size, activation='sigmoid', use_bias=True)(dec2)
  
  final_decoded_layer = []
  
  # split korbi
  
  reshaped = Lambda( lambda x: K.reshape(x, (-1, int(dec_output.shape[1]//3), 3)))(dec_output)
  
  final_decoded_layer_output = Softmax(axis=-1)(reshaped)
  
  
  autoencoder = Model(main_input, final_decoded_layer_output)
  #autoencoder = Model(main_input, dec_output)
  
  ## extra
  encoder = Model(main_input, enc4)
#   encoded_input = Input(batch_shape=(None, input_size))
#   decoder_layer = autoencoder.layers[-1]
#   decoder = Model(encoded_input, decoder_layer(encoded_input))
  
  return autoencoder, encoder  #, decoder, decoder_layer, encoded_input

In [0]:
autoencoder, encoder = get_model(input_size=495)

autoencoder.summary()

W0804 01:24:43.518822 140328488273792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 01:24:43.534093 140328488273792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 01:24:43.535886 140328488273792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 495)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 330)               163680    
_________________________________________________________________
dense_2 (Dense)              (None, 247)               81757     
_________________________________________________________________
dense_3 (Dense)              (None, 198)               49104     
_________________________________________________________________
dense_4 (Dense)              (None, 165)               32835     
_________________________________________________________________
dense_5 (Dense)              (None, 198)               32868     
_________________________________________________________________
dense_6 (Dense)              (None, 247)               49153     
__________

In [0]:
# https://datascience.stackexchange.com/questions/25029/custom-loss-function-with-additional-parameter-in-keras
def custom_accuracy_function(mask):
    # mask: ekta binary mask e.g [1,0,1,0,1,1,1]
    # 0 gula missing data represent kore
    def custom_accuracy(y_true, y_predict):
        y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')
        y_true_labels = K.cast(K.argmax(y_true, axis=2), 'float32')
        is_same = K.cast(K.equal(
            y_true_labels, y_pred_labels), 'float32')
        num_same = K.cast(K.sum(is_same * mask, axis=1), 'float32')
        lengths = K.cast(K.sum(mask, axis=1), 'float32')
        return K.mean(num_same / lengths, axis=0)
      
    return custom_accuracy
  
def custom_loss_function(mask):
    def custom_loss(y_true, y_predict):
        y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')
        y_true_labels = K.cast(K.argmax(y_true, axis=2), 'float32')
        is_same = K.cast(K.equal(
            y_true_labels, y_pred_labels), 'float32')
        num_same = K.cast(K.sum(is_same * mask, axis=1), 'float32')
        lengths = K.cast(K.sum(mask, axis=1), 'float32')
        return K.cast(1-K.mean(num_same / lengths, axis=0), 'float32')
      
    return custom_loss

In [0]:
print(Train_data_X_arr.shape)
mask = np.sum(Train_data_X_arr, axis=-1)#y_true

(7785, 165, 3)


In [0]:
print(mask.shape)
#print(mask)
#mask = np.reshape(mask, (mask.shape[0], mask.shape[1], 1))
# positions_seq = np.repeat([positions_seq],traindata.shape[0], axis=0)
# print(positions_seq.shape)
#print(mask.shape)
#mask = np.repeat(mask, 3, axis=-1)
#mask.shape

(7785, 165)


In [0]:
# Train_data_X_with_mask = np.asarray([Train_data_X_arr, mask])
# Train_data_X_with_mask = [Train_data_X_arr, mask]
# Train_data_X_with_mask = np.concatenate((Train_data_X_arr, mask), axis=-1)

In [0]:
#Train_data_X_with_mask.shape

In [0]:
"""def custom_accuracy_function(y_true_with_mask, y_predict):
    y_true_with_mask.shape
    y_true = y_true_with_mask[:, :3]
    mask = y_true_with_mask[:, 3]
    y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')
    y_true_labels = K.argmax(y_true, axis=2)
    y_true_labels = K.cast(y_true_labels, 'float32')
    is_same = K.cast(K.equal(
        y_true_labels, y_pred_labels), 'float32')
    num_same = K.sum(is_same * mask, axis=1)
    lengths = K.sum(mask, axis=1)
    return K.mean(num_same / lengths, axis=0)"""

"def custom_accuracy_function(y_true_with_mask, y_predict):\n    y_true_with_mask.shape\n    y_true = y_true_with_mask[:, :3]\n    mask = y_true_with_mask[:, 3]\n    y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')\n    y_true_labels = K.argmax(y_true, axis=2)\n    y_true_labels = K.cast(y_true_labels, 'float32')\n    is_same = K.cast(K.equal(\n        y_true_labels, y_pred_labels), 'float32')\n    num_same = K.sum(is_same * mask, axis=1)\n    lengths = K.sum(mask, axis=1)\n    return K.mean(num_same / lengths, axis=0)"

In [0]:
best_model_file = './ae_for_imputation'
checkpoint = callbacks.ModelCheckpoint(best_model_file, monitor='binary_crossentropy', verbose=1, save_best_only=True, mode='min')

In [0]:
print(Train_data_X_reshaped.shape)

input_size = Train_data_X_reshaped.shape[0]
print(input_size)

(7785, 495)
7785


In [0]:
# https://datascience.stackexchange.com/questions/25029/custom-loss-function-with-additional-parameter-in-keras
#autoencoder.compile(optimizer='adam', loss)

#adam = Adam(lr=0.0001)

autoencoder.compile(optimizer='adam',
              loss='binary_crossentropy', #https://github.com/LucaAngioloni/ProteinSecondaryStructure-CNN/blob/master/dataset.py
              sample_weight_mode='temporal',
              metrics=[custom_accuracy_function(mask=mask)])

W0804 01:24:43.987039 140328488273792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0804 01:24:44.014633 140328488273792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0804 01:24:44.020668 140328488273792 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
autoencoder.fit(x=Train_data_X_reshaped, y=Train_data_X_arr,batch_size=input_size,
        epochs=1000,
        shuffle=True,
        validation_split = 0.0,
        sample_weight=mask,
        verbose = 1)

Epoch 1/1000
7785/7785 [==============================] - 0s 14us/step - loss: 0.4145 - custom_accuracy: 0.7580
Epoch 2/1000
7785/7785 [==============================] - 0s 16us/step - loss: 0.4145 - custom_accuracy: 0.7579
Epoch 3/1000
7785/7785 [==============================] - 0s 14us/step - loss: 0.4145 - custom_accuracy: 0.7581
Epoch 4/1000
7785/7785 [==============================] - 0s 15us/step - loss: 0.4145 - custom_accuracy: 0.7583
Epoch 5/1000
7785/7785 [==============================] - 0s 14us/step - loss: 0.4145 - custom_accuracy: 0.7577
Epoch 6/1000
7785/7785 [==============================] - 0s 14us/step - loss: 0.4145 - custom_accuracy: 0.7581
Epoch 7/1000
7785/7785 [==============================] - 0s 14us/step - loss: 0.4145 - custom_accuracy: 0.7579
Epoch 8/1000
7785/7785 [==============================] - 0s 14us/step - loss: 0.4145 - custom_accuracy: 0.7585
Epoch 9/1000
7785/7785 [==============================] - 0s 14us/step - loss: 0.4145 - custom_accuracy:

In [0]:
imputed_position_mask = 1 - mask

In [0]:
imputed_position_mask.shape

(7785, 165)

In [0]:
y_pred = autoencoder.predict(Train_data_X_reshaped)

In [0]:
def direct_similarity_check_for_imputation(sample_no,Y_tree, Y_hat_tree, imputed_position_mask_for_tree, verbose=0):
#   ck = 0
  mismatch_count = 0
  print("checking tree ->" + str(sample_no))
  for i in range(len(Y_hat_tree)):
    if imputed_position_mask_for_tree[i]==1:
      if(verbose == 1):
        print(i, '>', Y_hat_tree[i], Y_tree[i], imputed_position_mask_for_tree[i])
      if(np.argmax(Y_hat_tree[i]) != np.argmax(Y_tree[i])):
        #print('mismatch at:', i, Y_hat_tree[i], Y_tree[i])
        mismatch_count += 1
  #       ck = 1
  #   if ck == 0:
  #     print('>>Fully matched.')
  #     return True
  #   return False
  return mismatch_count

In [0]:
mismatch_count = 0
temp = 0
for test_sample in range(input_size):
  #print('sample:', test_sample)
  temp = direct_similarity_check_for_imputation(test_sample,Train_data_Y_arr[test_sample], y_pred[test_sample], imputed_position_mask_for_tree=imputed_position_mask[test_sample], verbose=0)
  mismatch_count += temp
  print("number of mismatch in this tree -> " + str(temp))
  
print(mismatch_count)

temp2 = np.sum(imputed_position_mask)  
#acc = (1-mismatch_count/(np.size(Train_data_Y_arr,1)*np.size(Train_data_Y_arr,0)))
acc = (1-mismatch_count/temp2)
percent_acc =  acc*100
print(str(percent_acc) + "%")


In [0]:
print(mask.shape)
print(mask.shape[0]*mask.shape[1])
temp2 = np.sum(imputed_position_mask)  
print(temp2)
# mask_temp1 = mask[:int(mask.shape[0]/3)]
# print(mask_temp1.shape)

# autoencoder.train_on_batch(x=Train_data_X_reshaped, y=Train_data_X_arr,sample_weight=mask_temp1)

(7785, 165)
1284525
175783.0


In [0]:
test_sample = 13
cnt=direct_similarity_check_for_imputation(test_sample,Train_data_Y_arr[test_sample], y_pred[test_sample], imputed_position_mask_for_tree=imputed_position_mask[test_sample], verbose=1)
print(cnt)

checking tree ->13
0


In [0]:
np.sum(imputed_position_mask[test_sample])

0.0